In [ ]:
# !pip install recbole

In [ ]:
import os
from pathlib import Path

os.chdir('/content/drive/MyDrive/hnm')
DATA_PATH = Path.cwd() / 'data'
RAW = DATA_PATH / 'raw'
PROCESSED = DATA_PATH / 'processed'
SUBMISSION = DATA_PATH / 'submission'

import pandas as pd
import numpy as np

from recbole.quick_start.quick_start import load_data_and_model

from tqdm import tqdm

In [ ]:
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file='gru4rec/saved/GRU4Rec-Mar-29-2022_12-34-37.pth',
)

20 Apr 16:45    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = /content/drive/MyDrive/hnm/data/processed/recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 50
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = None
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [9, 0, 1]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}
repeatable = True
metrics = ['MAP']
topk = [12]
valid_metric = MAP@12
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_FIELD = timestamp
seq_le

In [ ]:
ext_user_ids = dataset.id2token(dataset.uid_field, list(range(dataset.user_num)))[1:]

In [ ]:
import torch
from recbole.data.interaction import Interaction

def add_last_item(old_interaction, last_item_id, max_len=50):
    new_seq_items = old_interaction['item_id_list'][-1]
    if old_interaction['item_length'][-1].item() < max_len:
        new_seq_items[old_interaction['item_length'][-1].item()] = last_item_id
    else:
        new_seq_items = torch.roll(new_seq_items, -1)
        new_seq_items[-1] = last_item_id
    return new_seq_items.view(1, len(new_seq_items))

def predict_for_all_item(external_user_id, dataset, model):
    model.eval()
    with torch.no_grad():
        uid_series = dataset.token2id(dataset.uid_field, [external_user_id])
        index = np.isin(dataset[dataset.uid_field].numpy(), uid_series)
        input_interaction = dataset[index]
        test = {
            'item_id_list': add_last_item(input_interaction, 
                                          input_interaction['item_id'][-1].item(), model.max_seq_length),
            'item_length': torch.tensor(
                [input_interaction['item_length'][-1].item() + 1
                 if input_interaction['item_length'][-1].item() < model.max_seq_length else model.max_seq_length])
        }
        new_inter = Interaction(test)
        new_inter = new_inter.to(config['device'])
        new_scores = model.full_sort_predict(new_inter)
        new_scores = new_scores.view(-1, test_data.dataset.item_num)
        new_scores[:, 0] = -np.inf  # set scores of [pad] to -inf
    return torch.topk(new_scores, 10)

In [ ]:
from recbole.utils.case_study import full_sort_topk

external_user_ids = dataset.id2token(dataset.uid_field, list(range(dataset.user_num)))[1:]

topk_items = []
for internal_user_id in tqdm(list(range(dataset.user_num))[1:]):
    _, topk_iid_list = full_sort_topk([internal_user_id], model, test_data, k=12, device=config['device'])
    last_topk_iid_list = topk_iid_list[-1]
    external_item_list = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()
    topk_items.append(external_item_list)

100%|██████████| 223127/223127 [14:32<00:00, 255.81it/s]


In [ ]:
revtopk = []

for row in tqdm(topk_items):
    revrow = ['0'+elem for elem in row]
    revtopk.append(revrow)

100%|██████████| 223127/223127 [00:00<00:00, 408864.25it/s]


In [ ]:
external_item_str = [' '.join(x) for x in revtopk]


In [ ]:
d = {}

for i in tqdm(range(len(external_user_ids))):
    if external_user_ids[i] in d:
        print('Error')
        break
    
    d[external_user_ids[i]] = external_item_str[i]


100%|██████████| 223127/223127 [00:00<00:00, 622529.62it/s]


In [ ]:
enb = pd.read_csv(SUBMISSION / 'ensemble_b.csv')


In [ ]:
for i, elem in tqdm(enb.iterrows(), total=enb.shape[0]):
    if elem.customer_id in d:
        elem.prediction = d[elem.customer_id]

100%|██████████| 1371980/1371980 [01:14<00:00, 18508.50it/s]


In [ ]:
enb.to_csv(SUBMISSION / 'GRU4Rec.csv', index=False)